# `QPOEstimation` tutorial

In [ ]:
%matplotlib inline

import os
from pathlib import Path
from scipy.signal import periodogram

import matplotlib
import matplotlib.pyplot as plt

import QPOEstimation
from QPOEstimation.get_data import *
from QPOEstimation.likelihood import get_kernel, get_mean_model, get_gp_likelihood
from QPOEstimation.prior import get_priors
from QPOEstimation.prior.gp import *
from QPOEstimation.utils import *

## Settings

Below are all possible options to set for runs. I tried my best to add some documentation. Generally, options are listed in `QPOEstimation.parse`.

### Basic data selection settings

These settings to point to the right data directory and correctly truncate the data if desired. The data is loaded from the local directories. Other data has to be added to these directories or loaded manually.

- `data_source`: Must be from `QPOEstimation.parse.DATA_SOURCES`. 
    - `"injection"`: Use "Injected", i.e. simulated data
    - `"giant_flare"`: Use data from the SGR 1806-20 Giant flare
    - `"solar_flare"`: Use solar flare data.
    - `"grb"`: Use GRB data.
    - `"magnetar_flare"`: Use magnetar flare data that is stored as time-tagged events.
    - `"magnetar_flare_binned"`: Use binned magnetar flare data.
    - `"hares_and_hounds"`: Use simulated data from Broomhall et al. 2019
- `run_modes`: Must be from `QPOEstimation.parse.RUN_MODES`. 
    - `"entire_segment"`: Use the entire possible data set.
    - `"select_time"`: Only use data between `start_time` and `end_time`.
    - `"from_maximum"`: truncate all data left of the maximum.
- `start_time`/`end_time`: When using `"select_time"` as run mode setting, where to truncate the data. 

In [ ]:
data_source = "magnetar_flare_binned"
run_mode = "entire_segment"
start_time = 0
end_time = 1

### Source specific data settings
In the following are settings specific to the `data_source` settings. Only the relevant one needs to be set. Settings for other sources are ignored.

#### Hares and Hounds settings
- `hares_and_hounds_id`: ID number of the desired data set.
- `hares_and_hounds_round`: `"HH2"` are the data sets we analysed for the paper. Otherwise use `"flare_sims"`, `"hh_change_bw"`, or `"HH_smooth"`.

In [ ]:
hares_and_hounds_id = "455290"
hares_and_hounds_round = "HH2"

#### Solar flare settings
- `solar_flare_folder`: The folder within `data/SolarFlare` to use.
- `solar_flare_id`: The label of the data file.

In [ ]:
solar_flare_folder = "goes"
solar_flare_id = "go1520130512"

#### GRB settings

These are just tailored to the few things we were looking at. It may make more sense to manually load in the data instead of using these functionalties.

- `grb_id`: The GRB name.
- `grb_label`: The label used for ASIM data.
- `grb_binning`: The binning for the Batse, Konus, or Swift data.
- `grb_detector`: `"batse`", `"konus"`, or `"swift"`
- `grb_energy_band`: The bands to use for swift data. Must be from `QPOEstimation.parse.GRB_ENERGY_BANDS`.

In [ ]:
grb_id = "090709A"
grb_label = ""
grb_binning = "1s"
grb_detector = "swift"
grb_energy_band = "all"

#### Magnetar flare settings

Magnetar data comes either as time-tagged event data (TTE) or already binned, hence the two `data_source` options.

- `magnetar_label`: The name of the magnetar.
- `magnetar_tag`: The filename of the specific flare of interest.
- `bin_size`: When using the unbinned `"magnetar_flare"` `data_source` setting, how finely to bin the data.
- `magnetar_subtract_t0`: Whether to set the time so that the first bin occurs at `t = 0`.
- `magnetar_unbarycentred_time`: For the TTE data whether to use the unbarycentered time.
- `rebin_factor`: For the binned data, makes the binning coarser. Useful if there are too few counts per bin.

In [ ]:
magnetar_label = "SGR_0501"
magnetar_tag = "080823478_lcobs"
bin_size = 0.001
magnetar_subtract_t0 = True
magnetar_unbarycentred_time = False
rebin_factor = 8

#### Giant flare settings

- `sampling_frequency`: The sampling frequency of the data in Hz.
- `period_number`: When using `run_mode = "sliding_window"`, which rotation period of the magnetar to use. These are separated by 7.56 seconds.
- `segment_length`: When using `run_mode = "sliding_window"`, the duration of the selected time to analyse.
- `segment_step`: When using `run_mode = "sliding_window"`, by how much to step the sliding window forward.
- `run_id`:  When using `run_mode = "sliding_window"`, how many segment steps to go forward from the start of the rotation period. 

In [ ]:
sampling_frequency = 64
period_number = 14
segment_length = 3.5
segment_step = 0.23625  # Requires 32 steps
run_id = 6

#### Injection data settings

These settings should point to the correct injection (i.e. simulated) data files, if they were created using `inject.py`.

- `injection_id`: The ID number of the injection file.
- `base_injection_outdir`: Where the injected data files are stored.
- `injection_file_dir`: The specific subdirectory where the injection file is stored
- `injection_mode`: Must be from `QPOEstimation.parse.MODES`. The kernel model that was used to create the injection data.
    - `"red_noise"`: The red noise kernel.
    - `"pure_qpo"`: The QPO kernel without any red noise.
    - `"qpo_plus_red_noise"`: The QPO kernel plus the red noise kernel.
    - `"qpo"`: The QPO kernel as defined in the celerite paper (we use `"qpo_plus_red_noise"` instead)
    - `"white_noise"`: No kernel, just constant white noise
    - `"matern32"`, `"matern52"`, `"exp_sine2"`, `"rational_quadratic"`,  `"exp_squared"`, `"exp_sine2_rn"`: Additional kernel functions that can be used with `george`
- `injection_likelihood_model`: The GP model used in the data created. Should be from `QPOEstimation.parse.LIKELIHOOD_MODELS`.
    - `"celerite"`: Use `celerite` to set up the likelihood, use in conjuncture with `"red_noise"`, `"pure_qpo"`, `"qpo_plus_red_noise"`, `"qpo"`, and `"white_noise"` kernels.
    - `"celerite_windowed"`: The same as `"celerite"` but with a non-stationary extension. Adds a start and end time parameter, between which the GP is applied. Outside this "window", white noise is applied.
    - `"george"`: Use `george` to set up the likelihood, use in conjuncture with `"matern32"`, `"matern52"`, `"exp_sine2"`, `"rational_quadratic"`,  `"exp_squared"`, and `"exp_sine2_rn"`.

In [ ]:
injection_id = 1
base_injection_outdir = "injections/injection"
injection_file_dir = "injection_files_pop"

injection_mode = "qpo_plus_red_noise"
injection_likelihood_model = "celerite"

### Prior settings

These are mostly set automatically from the data and you can leave them set at `None`. You shouldn't have to set these manually in most cases.

- `polynomial_max`: Maximal value of the polynomial coefficients when using a polynomial mean model.
- `amplitude_min`/`amplitude_max`: The minimum/maximum of the amplitude parameter(s).
- `offset_min`/`offset_max`: The minimum/maximum offset if we use a constant offset in the mean model.
- `sigma_min`/`sigma_max`: The minimum/maximum width parameter(s).
- `t_0_min`/`t_0_max`: The minimum/maximum flare peak location parameter(s).
- `min_log_a`/`max_log_a`: The minimum/maximum kernel log amplitude.
- `min_log_c_red_noise`/`max_log_c_red_noise`: The minimum/maximum red noise kernel log inverse decay time.
- `min_log_c_qpo`/`max_log_c_qpo`: The minimum/maximum QPO kernel log inverse decay time.
- `minimum_window_spacing`: Minimal GP duration for the `celerite_windowed` likelihood model.
- `band_minimum`/`band_maximum`: The minimum/maximum frequency.

In [ ]:
polynomial_max = 2
amplitude_min = None
amplitude_max = None
offset_min = None
offset_max = None
sigma_min = None
sigma_max = None
t_0_min = None
t_0_max = None

min_log_a = None
max_log_a = None
min_log_c_red_noise = None
min_log_c_qpo = None
max_log_c_red_noise = None
max_log_c_qpo = None
minimum_window_spacing = 0

band_minimum = None
band_maximum = None

### Settings for the kernel/mean model and the specific likelihood

- `recovery_mode`: Must be from `QPOEstimation.parse.MODES`, usually `red_noise` or `qpo_plus_red_noise`
    - `"red_noise"`: The red noise kernel.
    - `"pure_qpo"`: The QPO kernel without any red noise.
    - `"qpo_plus_red_noise"`: The QPO kernel plus the red noise kernel.
    - `"qpo"`: The QPO kernel as defined in the celerite paper (we use `"qpo_plus_red_noise"` instead)
    - `"white_noise"`: No kernel, just constant white noise
    - `"matern32"`, `"matern52"`, `"exp_sine2"`, `"rational_quadratic"`,  `"exp_squared"`, `"exp_sine2_rn"`: Additional kernel functions that can be used with `george`
- `likelihood_model`: Must be from `QPOEstimation.parse.LIKELIHOOD_MODELS`. In this case `"celerite"`, `"celerite_windowed"`, or `"george"`.
    - `"celerite"`: Use `celerite` to set up the likelihood, use in conjuncture with `"red_noise"`, `"pure_qpo"`, `"qpo_plus_red_noise"`, `"qpo"`, and `"white_noise"` kernels.
    - `"celerite_windowed"`: The same as `"celerite"` but with a non-stationary extension. Adds a start and end time parameter, between which the GP is applied. Outside this "window", white noise is applied.
    - `"george"`: Use `george` to set up the likelihood, use in conjuncture with `"matern32"`, `"matern52"`, `"exp_sine2"`, `"rational_quadratic"`,  `"exp_squared"`, and `"exp_sine2_rn"`.
- `background_model`: Must be from `QPOEstimation.parse.BACKGROUND_MODELS` or a constant float. 
    - `"skew_gaussian"`: The skewed Gaussian model.
    - `"skew_exponential"`: The skewed exponential model.
    - `"fred"`: The fast-rise exponential decay model.
    - `"mean"`: A constant mean value (no parameters).
    - `"polynomial"`: A polynomial up to the fourth order.
- `n_components`: How many of the `background_model` shapes we are using, e.g. `2` if we want to fit two `"skew_gaussian"` curves.
- `jitter_term`: Whether to add an additional free white noise term.
- `normalisation`: Normalises data so that `0 <= y <= 1` for all data points.
- `offset`: Whether to add a constant offset parameter to the mean model.

In [ ]:
recovery_mode = "qpo_plus_red_noise"
likelihood_model = "celerite"
background_model = "skew_gaussian"
n_components = 1
jitter_term = False
normalisation = False
offset = False

### Settings for the sampling procedure.

`sample = "rslice"` is faster but less accurate than `sample = "rwalk"`.

`nlive = 1000` should be sufficient for most setups.

In [ ]:
sample = "rslice"
nlive = 1000

### Whether to try to load an existing result file for these settings or try to resume from a run checkpoint.


In [ ]:
try_load = False
resume = False

### Some additional settings to make the result file labels distinguishable

In [ ]:
suffix = f"_{n_components}_{background_model}s"
band = f"{band_minimum}_{band_maximum}Hz"
truths = None
recovery_mode_str = recovery_mode
if jitter_term:
    recovery_mode_str += "_jitter"

### Call to the catch-all `get_data` function

If you want to load your own data or manually set output directories and labels, this is the piece of code to replace.

In [ ]:
times, y, yerr, outdir, label = get_data(
    data_source=data_source, band=band, segment_length=segment_length,
    sampling_frequency=sampling_frequency,
    period_number=period_number, run_id=run_id, segment_step=segment_step, start_time=start_time, end_time=end_time,
    run_mode=run_mode, recovery_mode=recovery_mode, recovery_mode_str=recovery_mode_str, likelihood_model=likelihood_model,
    magnetar_label=magnetar_label,  magnetar_tag=magnetar_tag, bin_size=bin_size,
    magnetar_subtract_t0=magnetar_subtract_t0, magnetar_unbarycentred_time=magnetar_unbarycentred_time,
    rebin_factor=rebin_factor, solar_flare_folder=solar_flare_folder, solar_flare_id=solar_flare_id,
    grb_id=grb_id, grb_binning=grb_binning, grb_detector=grb_detector, grb_label=grb_label, grb_energy_band=grb_energy_band,
    injection_file_dir=injection_file_dir, injection_mode=injection_mode, injection_id=injection_id,
    injection_likelihood_model=injection_likelihood_model, hares_and_hounds_id=hares_and_hounds_id,
    hares_and_hounds_round=hares_and_hounds_round, base_injection_outdir=base_injection_outdir
    )
label += suffix

### Whether to normalise the data to be between 0 and 1

In [ ]:
if normalisation:
    y = (y - np.min(y))/(np.max(y) - np.min(y)) * 1
    yerr = yerr/(np.max(y) - np.min(y))


### Plot the time-domain data

In [ ]:
plt.errorbar(times, y, yerr=yerr, fmt=".k", capsize=0, label="data")
plt.xlabel("time [s]")
plt.ylabel("flux")
plt.show()

### Plot the frequency-domain data

In [ ]:
fs = 1/(times[1] - times[0])
freqs, powers = periodogram(y, fs=fs, window="hann")
plt.loglog()
plt.step(freqs[1:], powers[1:])
plt.xlabel("frequency [Hz]")
plt.ylabel("Power [arb. units]")
plt.show()

### Get the priors

In [ ]:
mean_prior_bound_dict = dict(
    amplitude_min=amplitude_min,
    amplitude_max=amplitude_max,
    offset_min=offset_min,
    offset_max=offset_max,
    sigma_min=sigma_min,
    sigma_max=sigma_max,
    t_0_min=t_0_min,
    t_0_max=t_0_max,
)

kernel_prior_bound_dict = dict(
    min_log_a=min_log_a, max_log_a=max_log_a,
    min_log_c_red_noise=min_log_c_red_noise, 
    max_log_c_red_noise=max_log_c_red_noise,
    min_log_c_qpo=min_log_c_qpo, 
    max_log_c_qpo=max_log_c_qpo, 
    band_minimum=band_minimum, 
    band_maximum=band_maximum,
)

priors = get_priors(
    times=times, y=y, yerr=yerr, likelihood_model=likelihood_model, kernel_type=recovery_mode,
    model_type=background_model, polynomial_max=polynomial_max, minimum_spacing=minimum_window_spacing,
    n_components=n_components, offset=offset, jitter_term=jitter_term, 
    **kernel_prior_bound_dict, **mean_prior_bound_dict)

### Construct the mean model, kernel, and likelihood.

In [ ]:
mean_model = get_mean_model(
    model_type=background_model, n_components=n_components, y=y, offset=offset,
    likelihood_model=likelihood_model)
kernel = get_kernel(kernel_type=recovery_mode, jitter_term=jitter_term)
likelihood = get_gp_likelihood(mean_model=mean_model, kernel=kernel, times=times, y=y, yerr=yerr,
                               likelihood_model=likelihood_model)

### Store the `meta_data`

This information is stored in the result file and can be used to reconstruct the data and the model used in the inference process.

In [ ]:
meta_data = dict(
    kernel_type=recovery_mode, mean_model=background_model, times=times,
    y=y, yerr=yerr, likelihood_model=likelihood_model, truths=truths, n_components=n_components,
    offset=offset, jitter_term=jitter_term)

### Run the inference process

If `try_load = True`, try to load the result first. Otherwise, run the inference process using the usual `bilby` interface.

In [ ]:
result = None
if try_load:
    try:
        result = QPOEstimation.result.GPResult.from_json(outdir=f"{outdir}/results", label=label)
        result.outdir = f"{outdir}/results"
    except IOError:
        bilby.utils.logger.info("No result file found. Starting from scratch")
if result is None:
    Path(f"{outdir}/results").mkdir(parents=True, exist_ok=True)
    result = bilby.run_sampler(likelihood=likelihood, priors=priors, outdir=f"{outdir}/results",
                               label=label, sampler="dynesty", nlive=nlive, sample=sample,
                               resume=resume, result_class=QPOEstimation.result.GPResult,
                               meta_data=meta_data, save=True, gzip=False, nact=5, clean=True)

### Plotting functionalities

In [ ]:
result.plot_lightcurve()
result.plot_kernel()
result.plot_max_likelihood_psd()
result.plot_residual()
result.plot_frequency_posterior()
result.plot_period_posterior()
result.plot_duration_posterior()
# result.plot_all(paper_style=True)

### Location of additional information

`bilby` stores some information of the run in the result file. We can access many of these as properties, for example the log evidence

In [ ]:
print(result.log_evidence)